# Methods Overview: Scale Invariance Framework (Systems 1–9)

This notebook provides a guided, runnable tour of the repository’s core ideas and methods:

- Proving scale invariance in equilibrium sampling (Systems 1–3)
- A biological application: E. coli persistence as a two‑state thermodynamic switch
- An industry tool: TPU benchmarking via the Thermodynamic Integrity Score (System 9)

Throughout, we emphasize the central principle:

```
P(S; H, T) = P(S; α·H, α·T)  for all α > 0
```

Only the ratio H/T matters — scaling both by the same factor leaves the distribution unchanged.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scale_invariance import run_scale_invariance_test
# We'll plot directly here to avoid extra dependencies

# Optional imports guarded by try/except for sections below
try:
    from e_coli_persistence import run_persistence_prediction, plot_fel
except Exception:
    run_persistence_prediction = None
    plot_fel = None

try:
    from tpu_benchmark import benchmark_suite
except Exception:
    benchmark_suite = None

# Styling
plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['figure.dpi'] = 120


## 1) Scale Invariance: Three-Case Protocol (Systems 1–3)

We validate the invariance by comparing three distributions over the same state space:

- Case A (Original): (W, H, T)
- Case B (Control): (αW, αH, T)  — should differ
- Case C (Test): (αW, αH, αT) — should match Case A

We’ll generate these using `run_scale_invariance_test` and visualize them.


In [ ]:
N = 5
np.random.seed(42)
P_orig, P_scaled_E, P_test, params = run_scale_invariance_test(
    N=5, alpha=2.0, T0=1.0, num_samples=50000, seed=42
)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Case A
axes[0].bar(range(len(P_orig)), P_orig, alpha=0.7, color='#1f77b4')
axes[0].set_title('Case A: Original (W,H,T)')
axes[0].set_xlabel('State Index')
axes[0].set_ylabel('Probability')

# Case B (control)
axes[1].bar(range(len(P_scaled_E)), P_scaled_E, alpha=0.7, color='#ff7f0e')
axes[1].set_title('Case B: Control (αW,αH,T)')
axes[1].set_xlabel('State Index')

# Case C (test)
axes[2].bar(range(len(P_test)), P_test, alpha=0.7, color='#2ca02c')
axes[2].set_title('Case C: Test (αW,αH,αT)')
axes[2].set_xlabel('State Index')

plt.suptitle('Scale Invariance: Three-Case Comparison', y=1.05)
plt.tight_layout()
plt.show()

# Save publication-quality figure (optional)
try:
    fig.savefig('notebooks/viz/publication/scale_invariance_three_panel.png', dpi=300, bbox_inches='tight')
except Exception:
    pass


### Interpretation
- Case A vs Case C should be visually identical, demonstrating `P(S;H,T) = P(S; αH, αT)`.
- Case B differs and serves as a control to rule out trivial rescaling effects.


## 2) Biological Application: E. coli Persistence (Two-State Model)

We model the switch between Normal (S1) and Persister (S2) states as a Boltzmann policy over a simple two‑well free energy landscape. Environmental stress (e.g., antibiotic concentration) shifts the energy difference ΔG and barrier ΔG‡.

We use the helper `run_persistence_prediction` to parse an environment string and compute `P(Persister)`; `plot_fel` draws the double‑well landscape.


In [ ]:
if run_persistence_prediction is None:
    print("e_coli_persistence module unavailable in this environment.")
else:
    env = "Antibiotic A, 20uM, 37C"
    res = run_persistence_prediction(env)
    print({k: res[k] for k in ["temperature_K", "delta_G", "delta_G_dagger", "P_persister"]})

    fig, ax = plot_fel(res["fel_params"], res["temperature_K"])
    plt.show()
    try:
        fig.savefig('notebooks/viz/publication/fig_ecoli_fel.png', dpi=300, bbox_inches='tight')
    except Exception:
        pass


### Notes on the Method
- Probability of Persister: `P2 = 1 / (1 + exp(+ΔG/RT))` using ΔG = G2 − G1
- Stress mapping: ΔG decreases with antibiotic concentration (favoring S2), ΔG‡ increases (harder to switch without sufficient stress)
- Units: Energies in J/mol, T in Kelvin, `R = 8.314462618… J/mol/K`


## 3) TPU Integrity Benchmark (System 9)

The Thermodynamic Integrity Score (TIS) measures how closely hardware sampling replicates the reference Boltzmann distribution. Higher TIS indicates better hardware fidelity.

Below we optionally run a lightweight benchmark suite. If specialized backends aren’t available, it falls back to NumPy.


In [ ]:
if benchmark_suite is None:
    print("tpu_benchmark module unavailable in this environment.")
else:
    N = 5
    np.random.seed(0)
    W = np.random.randn(N, N)
    W = (W + W.T)/2
    H = np.random.randn(N)

    results = benchmark_suite(W, H, T0=1.0, alpha=2.0, num_samples=20000)
    # Print summary with TIS and D_KL if present
    for name, r in results.items():
        attrs = {k: getattr(r, k) for k in dir(r) if not k.startswith('_')}
        tis = attrs.get('tis', None)
        dkl = attrs.get('d_kl', None)
        print(f"{name:>15} | TIS: {tis} | D_KL: {dkl}")


## 4) Methods Summary and Pointers

- Equilibrium Invariance: `scale_invariance.py` (Systems 1–3) provides generators and tests; visualizations in `viz/scale_invariance_plots.py`.
- Biological Digital Twin: `e_coli_persistence.py` implements a two‑state free energy model and plotting helpers.
- Materials (System 8): See `materials_invariance.py` and the semiconductor `notebooks/Fabrication_Digital_Twin.ipynb`.
- TPU Benchmark (System 9): `tpu_benchmark.py` plus plots in `viz/tpu_benchmark_plots.py`.

You can explore additional figures under `notebooks/viz/publication/` and `viz/`.


### Acknowledgements
This work connects rigorous thermodynamic theory with practical engineering across many scales — from biological cells to industrial hardware.
